# get variables information from checkpoint

reference:
    - https://github.paypal.com/ART-INFRA/PySolCat/blob/master/SolCatOnlineVarDemo.ipynb

In [2]:
import pandas as pd

from PySolCat import SolCatOnlineVarLoader
from PySolCat import OutputTransform

onlineVarLoader = SolCatOnlineVarLoader('PAZ') 
ret = onlineVarLoader.get_online_var_by_cp_status(checkpoint='ConsolidatedFunding', status=['audit clean', 'implemented'])
var_info = pd.DataFrame(OutputTransform.convertToDict(ret))

In [3]:
var_info.head(5)

,_variable_name,_variable_type,return_type,value_type,variable_status,update_time,source,release_date,check_point,check_points,used_models,check_point_status
0,s09_exit_eg_oc_short_hist,component,double,continuous,audit clean,2022-11-22 22:30:24,rtcs,2020-06-24 00:00:00,"{'audit_clean_date': '2020-07-22 00:00:00', 'c...","[ConsolidatedFunding, WithdrawalAttempt, Fundi...",[],"[{'audit_clean_date': '2020-07-22 00:00:00', '..."
1,slr_r0_bizintl_builtwith_active_tag_count_java...,component,string,continuous,implemented,2022-11-22 22:22:16,rtcs,2018-06-12 00:00:00,"{'audit_clean_date': None, 'check_point_name':...","[ConsolidatedFunding, EvalUDOnboarding]",[],"[{'audit_clean_date': None, 'check_point_name'..."
2,FEEL_LAYER1_GIBBERISH_RISKY.model_solutionType,component,double,continuous,audit clean,2022-11-22 22:23:23,guanjia-rtcs,2022-09-08 08:07:50,"{'audit_clean_date': '2022-09-09 10:09:13', 'c...",[ConsolidatedFunding],[],"[{'audit_clean_date': '2022-09-09 10:09:13', '..."
3,idm_cc_hash_ace_is_cc_pre_auth_s_cnt_sw72_24h,component,string,continuous,audit clean,2022-11-22 22:38:27,rtcs,2022-06-28 13:26:00,"{'audit_clean_date': '2022-08-29 01:01:22', 'c...",[ConsolidatedFunding],[],"[{'audit_clean_date': '2022-08-29 01:01:22', '..."
4,slr_r0_bizintl_alexa_page_views_per_user_7_days,component,string,continuous,implemented,2022-11-22 22:39:19,rtcs,2018-06-12 00:00:00,"{'audit_clean_date': None, 'check_point_name':...","[ConsolidatedFunding, EvalUDOnboarding]",[],"[{'audit_clean_date': None, 'check_point_name'..."


# load clean variable list

In [ ]:
# load clean features
working_dir = ''
clean_features_file = "{}/assets/clean_features_{}.txt".format(working_dir, datetime.today().strftime("%Y%m%d"))

if os.path.exists(clean_features_file):
    print(f"load clean features from {clean_features_file}")
    clean_features = read_by_line(clean_features_file)

else:
    print(f"{clean_features_file} does not exist, load from SolCat")
    from PySolCat import SolCatOnlineVarLoader

    onlineVarLoader = SolCatOnlineVarLoader('PAZ')
    available_vars = onlineVarLoader.get_online_var_by_cp_status(checkpoint='ConsolidatedFunding', status=['audit clean', 'implemented'])
    clean_features = [var.__dict__['_variable_name'] for var in available_vars]

    with open(clean_features_file, 'w') as f:
        f.write("\n".join(clean_features))


# build daily solcat info

In [ ]:
def load_var_info(checkpoint_name='ConsolidatedFunding',
                  root_data_save_dir='/projects/gds-focus/data/guxia/UCC24/asset/solcat_var/',
                  status=['audit clean', 'implemented']):
    import os
    from datetime import datetime
    import pandas as pd
    from PySolCat import SolCatOnlineVarLoader
    from PySolCat import OutputTransform

    run_dt = datetime.now()
    var_solcat_save_path = f"{root_data_save_dir}/{checkpoint_name}_{run_dt.strftime('%Y%m%d')}.csv"
    os.makedirs(os.path.dirname(var_solcat_save_path), exist_ok=True)

    if not os.path.exists(var_solcat_save_path):
        print('daily var info not found, building from solcat')
        loader = SolCatOnlineVarLoader('PAZ')
        ret = loader.get_online_var_info_by_cp(checkpoint=checkpoint_name)
        var_info = pd.DataFrame(OutputTransform.convertToDict(ret))
        var_info = var_info.rename(columns={
            '_variable_name': 'variable',
            '_variable_type': 'variable_type',
        })
        var_info.to_csv(var_solcat_save_path, index=False)

        print(f'var info saved to {var_solcat_save_path}')
    else:
        var_info = pd.read_csv(var_solcat_save_path)
        print(f'loading var info from {var_solcat_save_path}')
        
    if len(status) > 0:
        var_info = var_info[var_info['variable_status'].isin(status)]
        print(f'after filter status, var_info shape: {var_info.shape}')
    
    return var_info